In [12]:
from decomposition import Decomposition
from dashboard import Dashboard
from utils import *
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotting import Radar, TimePlot, Brain, Spectre
import numpy.linalg as la
from matplotlib import cm

In [3]:
dcp1 = Decomposition()
dcp1.data = dcp1._extract_data(['/Volumes/GoogleDrive/My Drive/MIP-lab/data/Rest1LR/Rest1LR_Sub101107_Glasser.mat'])
dcp1.run()

dcp2 = Decomposition()
dcp2.data = dcp2._extract_data(['/Volumes/GoogleDrive/My Drive/MIP-lab/data/Rest1LR/Rest1LR_Sub103414_Glasser.mat'])
dcp2.run()

dcp = [dcp1, dcp2]

In [38]:
dcp1.df[['mode','value','damping_time','period']]

,mode,value,damping_time,period
0,1,0.958870+0.018672j,17.220884,232.342126
1,2,0.955709+0.000000j,15.893338,inf
2,3,0.921934+0.024022j,8.895254,173.661181
3,4,0.911410+0.066126j,7.987802,62.461498
4,5,0.911910+0.000000j,7.807895,inf
...,...,...,...,...
185,186,-0.094411-0.042093j,0.317258,1.661857
186,187,-0.029306-0.044660j,0.245764,2.102644
187,188,0.041943+0.000000j,0.227026,inf
188,189,-0.019579-0.025974j,0.210176,2.040800


## Testing Spectre

In [4]:
dcp1.df[['mode','value']]
ndf1 = pd.DataFrame({'Mode':dcp1.df['mode'], 'Value':np.abs(dcp1.df['value']), 'Group':[1 for i in range(dcp1.df.shape[0])]})
ndf2 = pd.DataFrame({'Mode':dcp2.df['mode'], 'Value':np.abs(dcp2.df['value']), 'Group':[2 for i in range(dcp2.df.shape[0])]})

Ndf = pd.concat([ndf1, ndf2])

## Testing Radar

In [3]:
df1 = pd.DataFrame({'mode': dcp1.df['mode'], 'group': [1 for i in range(dcp1.df.shape[0])],
                    'strength_real': dcp1.df['strength_real'], 'strength_imag': dcp1.df['strength_imag']}) \
    if dcp1.df is not None else None
df2 = pd.DataFrame({'mode': dcp2.df['mode'], 'group': [2 for i in range(dcp2.df.shape[0])],
                    'strength_real': dcp2.df['strength_real'], 'strength_imag': dcp2.df['strength_imag']}) \
    if dcp2.df is not None else None

networks = dcp1.df['networks'][0] if dcp1.df is not None else None
networks = dcp2.df['networks'][0] if dcp2.df is not None else None

df = pd.concat([df1, df2])

## Testing Brain

In [35]:
def _filter_brain(order):

    mode1 = dcp1.df.loc[order - 1][['intensity', 'conjugate']] if dcp1.df is not None else None
    mode2 = dcp2.df.loc[order - 1][['intensity', 'conjugate']] if dcp1.df is not None else None

    return 'glasser', mode1, mode2, order

def _figure(self):
    # Analysis
    if self.mode2 is None:

        rows, intensity = self._get_intensity([self.mode1])

        labels = ['Real'] if len(rows) == 1 else ['Real', 'Imaginary']

    # Comparison
    else:

        rows, intensity = self._get_intensity([self.mode1, self.mode2])

        if len(rows) == 2:
            labels = ['G1 Real', 'G2 Real']
        elif len(rows) == 3:
            if self.mode1.conjugate:
                labels = ['G1 Real', 'G1 Imaginary','G2 Real']
            else:
                labels = ['G1 Real', 'G2 Real','G2 Imaginary']
        else:
            labels = ['G1 Real', 'G1 Imaginary', 'G2 Real','G2 Imaginary']

    fig = make_subplots(rows=len(rows), cols=1,
                        # shared_xaxes=True,  # specs=[[{}], [{}]] if len(rows) == 2 else [[{}]],
                        horizontal_spacing=0.05,
                        vertical_spacing=0.05)

    for row in rows:

        for id in range(1, list(self.atlas['.id'])[-1] + 1):
            roi = self.atlas[self.atlas['.id'] == id]
            x = roi['.long']
            y = roi['.lat']
            region = roi['region'].iloc[0] if roi['region'].size != 0 else None

            if type(region) == str:

                indice = roi['.roi'].iloc[0] - 1

                if roi['hemi'].iloc[0] == 'right':
                    indice += int(intensity[0].shape[0] / 2)

                val = intensity[row - 1][indice]

                col = 'rgb({0},{1},{2})'.format(*cm.coolwarm(val)[:3])
            else:
                col = 'black'

            fig.add_trace(go.Scatter(x=x,
                                     y=y,
                                     fill='toself',
                                     mode='lines',
                                     line=dict(color='black', width=0.5),
                                     fillcolor=col,
                                     name=region if region else None,
                                     hoverinfo=None),
                          row=row, col=1)

        fig.add_trace(go.Scatter(x=[300, 1100], y=[-25, -25],
                                 text=['left', 'right'],
                                 mode='text'),
                      row=row, col=1)

    axis_config = dict(showgrid=False, showline=False, visible=False, ticks='',
                       showticklabels=False, zeroline=False, showspikes=False)

    fig.update_layout(paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
                      showlegend=False, title_text='Mode {}'.format(self.order if self.order is not None else ''),
                      height=150+len(rows)*200)

    for i, label in enumerate(labels):

        fig.layout['xaxis' + str(i + 1)].update(axis_config)
        fig.layout['yaxis' + str(i + 1)].update({**axis_config,
                                                 **dict(scaleanchor='x' + str(i + 1), scaleratio=1,
                                                        title=labels[i], visible=True)})
    fig.add_trace(go.Scatter(
                    x=[200,200],
                    y=[-20,-20],
                    marker=dict(
                        size=0.01,
                        opacity=1,
                        cmax=0.1,
                        cmin=-0.1,
                        color=[-0.1,0.1],
                        colorbar=dict(
                            title="Activity",
                            len=.5
                        ),
                        colorscale="RdBu",
                        reversescale=True
                    ),
                    mode="markers"))

    return fig

Brain.figure = _figure

In [36]:
order = 1
b = Brain(*_filter_brain(order))
fig = b.figure()
fig

## Testing TimePlot

In [3]:
df1 = pd.DataFrame({'Mode': dcp1.df['mode'], 'Activity': dcp1.df['activity'],
                    'Group': ['Group 1' for i in range(dcp1.df.shape[0])]}) \
    if dcp1.df is not None else None
df2 = pd.DataFrame({'Mode': dcp2.df['mode'], 'Activity': dcp2.df['activity'],
                    'Group': ['Group 2' for i in range(dcp2.df.shape[0])]}) \
    if dcp2.df is not None else None

DF = pd.concat([df1, df2])

In [4]:
t = TimePlot(DF)

In [7]:
fig = t.figure()
fig.layout['height'] = 900
fig